In [1]:
import numpy as np

import torch
import pytorch_lightning as pl

from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Subset

from deeptime.data import BaseDataset, UCRDataset
from deeptime.models.representation import LinearAutoEncoder

C:\Users\medei\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = UCRDataset(name='FordA', split='train')
len(train_dataset)

3601

In [3]:
test_dataset = UCRDataset(name='FordA', split='test')
len(test_dataset)

1320

In [4]:
unique_labels = torch.unique(train_dataset[:][1])
unique_labels

tensor([-1.,  1.])

In [5]:
for label in unique_labels:
    x_train, y_train = train_dataset[label == train_dataset[:][1]]

    filtered_dataset = BaseDataset(
        x=np.array(x_train.tolist()),
        y=np.array(y_train.tolist())
    )

    train_loader = DataLoader(filtered_dataset, batch_size=16)

    autoencoder = LinearAutoEncoder(input_dim=x_train.shape[-1], latent_dim=32)
    trainer = pl.Trainer(max_epochs=1000, accelerator='gpu', devices=-1)

    trainer.fit(model=autoencoder, train_dataloaders=train_loader)
    break

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\medei\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | e    | Sequential | 516 K 
1 | d    | Sequential | 516 K 
------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.128     Total estimated model params size (MB)
C:\Users\medei\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasi

Epoch 109:   6%|██▉                                              | 7/116 [00:00<00:01, 88.61it/s, loss=0.335, v_num=10]

C:\Users\medei\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [6]:
import matplotlib.pyplot as plt

x_hat, z = autoencoder(train_dataset[0][0].view(1, -1))

plt.plot(list(range(500)), train_dataset[0][0].view(-1).tolist())
plt.plot(list(range(500)), x_hat.view(-1).tolist(), color='red')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_mm)